In [1]:
%matplotlib inline
##%matplotlib notebook
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10,8)
import pylab as plt
from astrometry.libkd.spherematch import *
from astrometry.util.fits import *
import numpy as np
from astrometry.util.starutil_numpy import *
from astrometry.util.plotutils import *
from glob import glob

In [2]:
T=fits_table('~/cosmo/data/wise/merge/merge_p1bm_frm/WISE-l1b-metadata-neowiser.fits')
T.cut(np.argsort(T.mjd))

In [ ]:
I=np.flatnonzero(np.abs(T.mjd - 56708) < 5.)
T.ecl_l,T.ecl_b = radectoecliptic(T.ra0, T.dec0)

In [ ]:
plt.plot(T.ra0[I], T.dec0[I], 'b.')

In [ ]:
plt.plot(T.ecl_l[I], T.ecl_b[I], 'b.')

In [ ]:
J=np.flatnonzero((np.abs(T.mjd - 56708) < 5.) * (T.ecl_b < -89))

In [ ]:
J1=np.flatnonzero((np.abs(T.mjd - 56708) < 5.) * (T.ecl_b < -89.5) * (T.ecl_l < 180))
plt.plot(T.ecl_l[J1], T.ecl_b[J1], 'b.')
len(np.unique(T.scan_id[J1])), len(J1)

In [ ]:
J1=np.flatnonzero((np.abs(T.mjd - 56708) < 5.) * (T.ecl_b < -89.3))

In [ ]:
T.scan_id[J1[:10]]

In [ ]:
plt.plot(T.mjd[J1])

In [ ]:
from unwise_coadd import *

In [ ]:
j = T[J1[0]]
fn = get_l1b_file('/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/', j.scan_id, j.frame_num, 1)
print(fn)
j1 = T[J1[1]]
fn1 = get_l1b_file('/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/', j1.scan_id, j1.frame_num, 1)
print(fn1)
j = T[J1[2]]
fn2 = get_l1b_file('/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/', j.scan_id, j.frame_num, 1)
print(fn2)

In [ ]:
img = fitsio.read(fn)
img[np.isfinite(img)==False] = 15.
plt.imshow(img, vmin=15, vmax=50)

In [ ]:
from astrometry.util.util import Tan, Sip

wcs = get_coadd_tile_wcs(90.0, -66.56)

imgwcs = Sip(fn)
imgwcs1 = Sip(fn1)

In [ ]:
imgwcs.pixel_scale(), imgwcs.shape, 1016*2.75/3600.

In [ ]:
from astrometry.util.resample import *
Yo,Xo,Yi,Xi,[rimg] = resample_with_wcs(wcs, imgwcs, [img], 3)

In [ ]:
coimg = np.zeros(wcs.shape, np.float32)
coimg[Yo,Xo] = rimg

In [ ]:
plt.imshow(coimg, vmin=15, vmax=50);

In [ ]:
img1 = fitsio.read(fn1)
img1[np.isfinite(img1)==False] = 15.
print(np.percentile(img1, [50,98]))
Yo,Xo,Yi,Xi,[rimg] = resample_with_wcs(wcs, imgwcs1, [img1], 3)
coimg1 = np.zeros(wcs.shape, np.float32)
coimg1[Yo,Xo] = rimg
plt.imshow(coimg1, vmin=15, vmax=50, interpolation='nearest', origin='lower');
plt.imsave('coimg0.png', coimg, vmin=15, vmax=50, origin='lower');
plt.imsave('coimg1.png', coimg1, vmin=15, vmax=50, origin='lower');

In [ ]:
plt.imshow(coimg1 - coimg, vmin=-10, vmax=10);

In [ ]:
scans = np.unique(T.scan_id[J1])
print(len(scans), 'scans')
#scans = [scans[i] for i in ]
doscans = [0,1,19,20,40,41,99,100]
coimgs = {}
for iscan,scan in enumerate(scans):
    if not iscan in doscans:
        continue
    print('Scan', iscan)
    K = J1[T.scan_id[J1] == scan]
    coimg = np.zeros(wcs.shape, np.float32)
    con = np.zeros(wcs.shape, np.uint8)
    for k in K:
        frame = T[k]
        fn = get_l1b_file('/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/', frame.scan_id, frame.frame_num, 1)
        print(fn)
        img = fitsio.read(fn)
        ok = np.isfinite(img)
        med = np.percentile(img[ok], 50)
        img[ok==False] = med
        imgwcs = Sip(fn)
        #ok,x,y = imgwcs.radec2pixelxy(90.64659110189505, -66.72252927611274)
        ok,x,y = imgwcs.radec2pixelxy(90.74254187941929, -66.69918917807308)
        print('->', int(x), int(y))
        Yo,Xo,Yi,Xi,[rimg] = resample_with_wcs(wcs, imgwcs, [img-med], 3)
        coimg[Yo,Xo] += rimg
        con[Yo,Xo] += 1
    coimg /= np.maximum(con, 1)
    coimgs[iscan] = coimg
    plt.imsave('coimg-%03i.png' % iscan, coimg, origin='lower', vmin=0, vmax=25)
    #if iscan == 19:
    #    break

In [ ]:
medimg = np.median(np.dstack(coimgs.values()), axis=2)
maximg = np.max(np.dstack(coimgs.values()), axis=2)

In [ ]:
plt.imshow(medimg, interpolation='nearest', origin='lower', vmin=0, vmax=25);
plt.axis([500,750,750,1000]);

In [ ]:
plt.imshow(maximg-medimg, interpolation='nearest', origin='lower', vmin=0, vmax=25);
plt.axis([500,750,750,1000]);

In [ ]:
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.measurements import label, find_objects


print(doscans)
coords = {
    0: (692,802),
    1: (641,837),
    19: (674,817),
    20: (621,849),
    40: (652,832),
    41: (599,864),
    99: (587,869),
    100: (531,896),
}
iscan = 100
plt.subplot(1,2,1)
plt.imshow(np.abs(coimgs[iscan]-medimg), interpolation='nearest', origin='lower', vmin=0, vmax=25);

det = gaussian_filter(coimgs[iscan]-medimg, 2.)
hot = (det > 5.)
blobs,nblobs = label(hot)
slices = find_objects(blobs)
xx,yy = np.zeros(nblobs, int), np.zeros(nblobs, int)
for i,(blob,slc) in enumerate(zip(blobs,slices)):
    sub = det[slc]
    #print('blob values within', i, np.unique(blobs[slc]))
    sub *= (blobs[slc] == (i+1))
    y0,x0 = slc[0].start, slc[1].start
    my,mx = np.unravel_index(np.argmax(sub), sub.shape)
    xx[i] = x0+mx
    yy[i] = y0+my

plt.axis([500,750,750,1000]);
plt.subplot(1,2,2)

x,y = coords[iscan]

closest = np.argmin(np.hypot(xx - x, yy - y))
plt.imshow(np.abs(coimgs[iscan]-medimg), interpolation='nearest', origin='lower', vmin=0, vmax=25);
plt.plot(xx, yy, 'ro')
plt.plot(xx[closest], yy[closest], 'mo', ms=20, mew=4, mfc='none')
print('Closest:', xx[closest], yy[closest])
plt.axvline(x, color='r')
#plt.axvline(x-5, color='r')
#plt.axvline(x+5, color='r')
plt.axhline(y, color='r')
#plt.axhline(y-5, color='r')
#plt.axhline(y+5, color='r')
plt.axis([x-50,x+50, y-50, y+50]);

In [ ]:
# Find out the specific L1b image containing the RA,Dec point in each scan.
fns = {}
scanwcs = {}
scandates = {}
scpos = {}
dirs = {}
for iscan in doscans:
    print('Scan', iscan)
    scan = scans[iscan]
    K = J1[T.scan_id[J1] == scan]
    for k in K:
        frame = T[k]
        fn = get_l1b_file('/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/', frame.scan_id, frame.frame_num, 1)
        print(fn)
        imgwcs = Sip(fn)
        ok,x,y = imgwcs.radec2pixelxy(90.74254187941929, -66.69918917807308)
        H,W = imgwcs.shape
        if x >= 1 and y >= 1 and x <= W and y <= H:
            print('->', int(x), int(y))
            fns[iscan] = fn
            scanwcs[iscan] = imgwcs    
            hdr = fitsio.read_header(fn)
            scandates[iscan] = hdr['DATE_OBS']
            scpos[iscan] = auinkm * np.array([hdr['ERTH2SCX'],hdr['ERTH2SCY'],hdr['ERTH2SCZ']])
            x,y = coords[iscan]
            r,d = wcs.pixelxy2radec(x+1, y+1)
            dirs[iscan] = radectoxyz(r,d)[0]
            break

In [ ]:
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
ax = plt.gcf().add_subplot(111, projection='3d')
for iscan in doscans:
    sc = scpos[iscan]
    d1 = scpos[iscan] + dirs[iscan]*1000.
    ax.plot3D([sc[0]], [sc[1]], [sc[2]], 'o')
    ax.plot3D([sc[0],d1[0]], [sc[1],d1[1]], [sc[2],d1[2]], '-')
ax.plot3D([0],[0],[0], 'ro')
plt.show()

In [ ]:
'''
SUN2SCX =    -0.81955005972593 / [AU] Sun-to-S/C vector X component, J2000
SUN2SCY =    0.505716170733614 / [AU] Sun-to-S/C vector Y component, J2000
SUN2SCZ =     0.21919071289506 / [AU] Sun-to-S/C vector Z component, J2000
SCVELX  = -0.00743308810283315 / [AU/day] SUN-to-S/C velocity vector X component
SCVELY  =  -0.0097310164738194 / [AU/day] SUN-to-S/C velocity vector Y component
SCVELZ  = -0.00445218519404931 / [AU/day] SUN-to-S/C velocity vector Z component
ERTH2SCX= 2.39242578775712E-06 / [AU] Earth-to-S/C vector X component
ERTH2SCY= 1.42680866487827E-05 / [AU] Earth-to-S/C vector Y component
ERTH2SCZ= -4.37456411757718E-05 / [AU] Earth-to-S/C vector Z component

GEOLON  =           275.157902 / [deg] S/C geographic longitude
GEOLAT  =           -71.497494 / [deg] S/C geographic latitude
GEOALT  =           533.749211 / [km] S/C altitude
''';

In [ ]:
fn1 = '/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/2a/46132a/269/46132a269-w1-int-1b.fits'
fn2 = '/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/4b/46134b/010/46134b010-w1-int-1b.fits'
fn19 = '/global/homes/d/dstn/cosmo/data/wise/merge/merge_p1bm_frm/2a/46172a/270/46172a270-w1-int-1b.fits'
wcs1 = Sip(fn1)
wcs2 = Sip(fn2)
wcs19 = Sip(fn19)
hdr1 = fitsio.read_header(fn1)
hdr2 = fitsio.read_header(fn2)
hdr19 = fitsio.read_header(fn19)
#ra1,dec1 = wcs.pixelxy2radec(690, 810)
#ra2,dec2 = wcs.pixelxy2radec(640, 840)
ra1,dec1 = wcs1.pixelxy2radec(593, 32)
ra2,dec2 = wcs2.pixelxy2radec(671, 674)
ra19,dec19 = wcs19.pixelxy2radec(623, 492)
print(hdr1['DATE_OBS'], hdr2['DATE_OBS'], hdr19['DATE_OBS'])

In [ ]:
auinkm = 1.5e8
sc1 = auinkm * np.array([hdr1['ERTH2SCX'],hdr1['ERTH2SCY'],hdr1['ERTH2SCZ']])
dir1 = radectoxyz(ra1,dec1)[0]
sc2 = auinkm * np.array([hdr2['ERTH2SCX'],hdr2['ERTH2SCY'],hdr2['ERTH2SCZ']])
dir2 = radectoxyz(ra2,dec2)[0]
sc19 = auinkm * np.array([hdr19['ERTH2SCX'],hdr19['ERTH2SCY'],hdr19['ERTH2SCZ']])
dir19 = radectoxyz(ra19,dec19)[0]

In [ ]:
print(np.sqrt(np.sum((sc1 - sc2)**2)) * 1.5e8)
dir1
sc1

In [ ]:
#%matplotlib notebook
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
ax = plt.gcf().add_subplot(111, projection='3d')
d1 = sc1 + dir1*1000.
d2 = sc2 + dir2*1000.
d19 = sc19 + dir19*1000.
ax.plot3D([sc1[0]], [sc1[1]], [sc1[2]], 'bo')
ax.plot3D([sc1[0],d1[0]], [sc1[1],d1[1]], [sc1[2],d1[2]], 'b-')
ax.plot3D([sc2[0]], [sc2[1]], [sc2[2]], 'go')
ax.plot3D([sc2[0],d2[0]], [sc2[1],d2[1]], [sc2[2],d2[2]], 'g-')
ax.plot3D([sc19[0]], [sc19[1]], [sc19[2]], 'mo')
ax.plot3D([sc19[0],d19[0]], [sc19[1],d19[1]], [sc19[2],d19[2]], 'm-')
ax.plot3D([0],[0],[0], 'ro')
plt.show()

In [ ]:
dir1, dir2

In [ ]:
sc1,sc2

In [ ]:
auinkm = 1.5e8
sc1km = sc1 * auinkm
sc2km = sc2 * auinkm
dists = np.linspace(0, 2500., 100)
seps = np.zeros_like(dists)
for i,d in enumerate(dists):
    seps[i] = np.sqrt(np.sum(((sc1km + d*dir1) - (sc2km + d*dir2))**2))

In [ ]:
%matplotlib inline
plt.plot(dists, seps, 'b-')
plt.show()

In [ ]:
min(seps)